In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [2]:
data = pd.read_csv("email.csv")

In [3]:
data.head(5)

,label,text
0,ham,Subject: enron methanol ; meter # : 988291\nth...
1,ham,"Subject: hpl nom for january 9 , 2001\n( see a..."
2,ham,"Subject: neon retreat\nho ho ho , we ' re arou..."
3,spam,"Subject: photoshop , windows , office . cheap ..."
4,ham,Subject: re : indian springs\nthis deal is to ...


In [4]:
data['label'].value_counts()

label
ham     3672
spam    1499
Name: count, dtype: int64

In [5]:
df_spam = data[data['label'] == 'spam']

In [6]:
df_ham = data[data['label'] == 'ham']

In [7]:
df_ham_reduced = df_ham.sample(df_spam.shape[0])

In [8]:
df_balanced = pd.concat([df_ham_reduced, df_spam])

In [9]:
df_balanced['label'].value_counts()

label
ham     1499
spam    1499
Name: count, dtype: int64

In [10]:
df_balanced['num_label'] = df_balanced['label'].apply(lambda x: 1 if x == 'spam' else 0)

In [11]:
df_balanced.sample(5)

,label,text,num_label
171,ham,Subject: re : e mail list for class\nplease re...,0
1692,ham,Subject: fw : guadalupe power partners\nfyi\n-...,0
357,ham,Subject: july pricing discrepancy : sell to te...,0
1407,spam,Subject: works wondder\ndear sir / madam .\nwe...,1
3741,spam,Subject: save up to 89 % on ink + no shipping ...,1


In [12]:
from sklearn.model_selection import train_test_split

In [13]:
x_train, x_test, y_train, y_test = train_test_split(df_balanced['text'], df_balanced['num_label'],stratify = df_balanced['num_label'])

C:\Users\Harshit Anand\.conda\envs\py35\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\Harshit Anand\.conda\envs\py35\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [14]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [15]:
input = tf.keras.layers.Input(shape = (), dtype = tf.string, name = 'text')
preprocessed_text = bert_preprocess(input)
outputs = bert_encoder(preprocessed_text)

In [16]:
l = tf.keras.layers.Dropout(0.1, name = "Dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

In [17]:
model = tf.keras.Model(inputs = [input], outputs = [l])

In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [19]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name = 'accuracy'),
    tf.keras.metrics.Precision(name = 'precision'),
    tf.keras.metrics.Recall(name = 'recall')
]
model.compile(optimizer = 'adam',
               loss = 'binary_crossentropy',
               metrics = METRICS)

In [20]:
model.fit(x_train, y_train, epochs = 10)

Epoch 1/10
71/71 [==============================] - 284s 4s/step - loss: 0.6519 - accuracy: 0.6174 - precision: 0.6252 - recall: 0.5863
Epoch 2/10
71/71 [==============================] - 274s 4s/step - loss: 0.5613 - accuracy: 0.7140 - precision: 0.7123 - recall: 0.7180
Epoch 3/10
71/71 [==============================] - 276s 4s/step - loss: 0.5138 - accuracy: 0.7478 - precision: 0.7553 - recall: 0.7331
Epoch 4/10
71/71 [==============================] - 276s 4s/step - loss: 0.4757 - accuracy: 0.7945 - precision: 0.7961 - recall: 0.7918
Epoch 5/10
71/71 [==============================] - 274s 4s/step - loss: 0.4584 - accuracy: 0.8025 - precision: 0.8080 - recall: 0.7936
Epoch 6/10
71/71 [==============================] - 274s 4s/step - loss: 0.4220 - accuracy: 0.8394 - precision: 0.8446 - recall: 0.8319
Epoch 7/10
71/71 [==============================] - 278s 4s/step - loss: 0.4077 - accuracy: 0.8545 - precision: 0.8639 - recall: 0.8416
Epoch 8/10
71/71 [==============================

In [21]:
y_predicted = model.predict(x_test)
y_predicted = y_predicted.flatten()
y_predicted = np.where(y_predicted > 0.5,1,0)
y_predicted

24/24 [==============================] - 93s 4s/step


array([1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,

In [22]:
sample_dataset = [
 "You can win a lot of money, register in the link below",
 "You have an iPhone 10, spin the image below to claim your prize and it will be delivered in your door step",
 "You have an offer, the company will give you 50% off on every item purchased.",
 "Hey Bravin, don't be late for the meeting tomorrow will start lot exactly 10:30 am",
 "See you monday, we have alot to talk about the future of this company ."
]

In [24]:
prediction = model.predict(sample_dataset)
prediction = np.where(prediction > 0.5, 1, 0)
prediction

1/1 [==============================] - 1s 606ms/step


array([[1],
       [1],
       [1],
       [0],
       [0]])